# LSTM (23 feature) ? 1/7/30 kunlik forecast

- MinMaxScaler (feature + target), 30-kunlik oyna -> 1-step model.
- 1/7/30 forecast: feature oynasi oxirgi qator qiymati bilan takrorlanadi (kelajak featurelari yo'q farazida).


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Ma'lumotlarni yuklash
p = Path('datasets/usd_rates_ready.csv')
df = pd.read_csv(p)
df.columns = [c.lower() for c in df.columns]
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

# Target = rate, date ishlatilmaydi, qolgan 23 ta ustun feature sifatida
feature_cols = [c for c in df.columns if c not in ['date', 'rate']]

# Outlierlarni IQR bilan yumshatish (faqat targetda)
def clip_iqr(s, k=1.5):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    lower, upper = q1 - k * iqr, q3 + k * iqr
    return s.clip(lower, upper)

df['rate'] = clip_iqr(df['rate'])

# Vaqt bo'yicha train/val/test (70/15/15)
n = len(df)
train_end = int(n * 0.70)
val_end = int(n * 0.85)
train_df = df.iloc[:train_end]
val_df = df.iloc[train_end:val_end]
test_df = df.iloc[val_end:]

# Bashorat ufqlari
horizons = [1, 7, 30]

from sklearn.preprocessing import MinMaxScaler
try:
    import tensorflow as tf
except ImportError:
    raise SystemExit("TensorFlow o'rnatilmagan: pip install tensorflow")

feat_scaler = MinMaxScaler()
target_scaler = MinMaxScaler()
features_scaled = feat_scaler.fit_transform(df[feature_cols])
target_scaled = target_scaler.fit_transform(df[['rate']]).ravel()

window = 30
horizon = 1
X_seq, y_seq = [], []
for i in range(len(df) - window - horizon + 1):
    X_seq.append(features_scaled[i:i+window, :])
    y_seq.append(target_scaled[i+window])
X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

n = len(X_seq)
train_end = int(n * 0.70)
val_end = int(n * 0.85)
X_train, y_train = X_seq[:train_end], y_seq[:train_end]
X_val, y_val = X_seq[train_end:val_end], y_seq[train_end:val_end]
X_test, y_test = X_seq[val_end:], y_seq[val_end:]

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(window, len(feature_cols))),
    tf.keras.layers.LSTM(64, return_sequences=False),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(horizon)
])
model.compile(optimizer='adam', loss='mae')

es = tf.keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, callbacks=[es], verbose=0)

val_pred = model.predict(X_val, verbose=0).ravel()
test_pred = model.predict(X_test, verbose=0).ravel()

val_true = target_scaler.inverse_transform(val_pred.reshape(-1, 1))
val_label = target_scaler.inverse_transform(y_val.reshape(-1, 1))
test_true = target_scaler.inverse_transform(test_pred.reshape(-1, 1))
test_label = target_scaler.inverse_transform(y_test.reshape(-1, 1))

mae_val = mean_absolute_error(val_label, val_true)
mape_val = mean_absolute_percentage_error(val_label, val_true)
mae_test = mean_absolute_error(test_label, test_true)
mape_test = mean_absolute_percentage_error(test_label, test_true)
print(f"Val MAE: {mae_val:.2f}, MAPE: {mape_val:.4f}")
print(f"Test MAE: {mae_test:.2f}, MAPE: {mape_test:.4f}")

# 1/7/30 kunlik forecast (feature oynasi oxirgi qiymat bilan)
last_window = features_scaled[-window:, :].copy()
future_preds = {}
for h in horizons:
    state = last_window.copy()
    preds = []
    for _ in range(h):
        x_in = state.reshape(1, window, len(feature_cols))
        next_scaled = model.predict(x_in, verbose=0)[0, 0]
        preds.append(next_scaled)
        # kelajak featurelari yo'q, shuning uchun oynani oxirgi qator bilan uzaytiramiz
        state = np.concatenate([state[1:], state[-1:]], axis=0)
    preds_inv = target_scaler.inverse_transform(np.array(preds).reshape(-1, 1)).ravel()
    future_dates = pd.date_range(df['date'].max() + pd.Timedelta(days=1), periods=h, freq='D')
    future_preds[f'h_{h}'] = pd.DataFrame({'date': future_dates, 'pred_rate': preds_inv})

future_preds['h_30'].head()
